# Imports & Settings

In [1]:
# Core tools
import joblib
import pandas as pd

# Explainer Dashboard
from explainerdashboard.custom import *
from explainerdashboard import RegressionExplainer, ExplainerDashboard

# Setting Ticker
Until functionality is built directly into the dashboard for switching between companies, the ticker to be analyzed will need to be manually specified here.

In [3]:
ticker = 'MSFT'

# Loading S&P 500 Data

In [34]:
snp = pd.read_csv('data/sp500.csv')
snp.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


# Loading Price History Data

In [33]:
prices = pd.read_csv(f'data/price_histories/{ticker}_history.csv', index_col='Date')
prices.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1986-03-13,0.088542,0.101563,0.088542,0.097222,0.061491,1.031789e+09
1986-03-14,0.097222,0.102431,0.097222,0.100694,0.063687,3.081600e+08
1986-03-17,0.100694,0.103299,0.100694,0.102431,0.064785,1.331712e+08
1986-03-18,0.102431,0.103299,0.098958,0.099826,0.063138,6.776640e+07
1986-03-19,0.099826,0.100694,0.097222,0.098090,0.062040,4.789440e+07


# Loading Model Data

In [4]:
data = pd.read_csv(f'data/preprocessed_data/{ticker}_preprocessed.csv')
data = data.drop(columns=['Report Date', 'Price Date', 'Open', 'High', 'Low', 'Adj Close', 'Volume'])
data.columns = [col.replace('.', '') for col in data.columns]
X = data.drop(columns=['Close'])
y = data.Close

In [5]:
model = joblib.load(f'models/{ticker}_model.joblib')
type(model)

xgboost.sklearn.XGBRFRegressor

In [6]:
model

XGBRFRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.2,
               colsample_bytree=0.2, gamma=50, gpu_id=-1,
               importance_type='gain', interaction_constraints='',
               learning_rate=0.5, max_delta_step=0, max_depth=4,
               min_child_weight=1, missing=nan, monotone_constraints='()',
               n_estimators=100, n_jobs=0, num_parallel_tree=100,
               objective='reg:squarederror', random_state=42, reg_alpha=0,
               scale_pos_weight=1, tree_method='exact', validate_parameters=1,
               verbosity=None)

# Explainer

In [7]:
explainer = RegressionExplainer(model, X, y, shap='tree')

Generating self.shap_explainer = shap.TreeExplainer(model)


# Custom Layout

## Overview Tab

In [37]:
class OverviewTab(ExplainerComponent):
    def __init__(self, explainer, name=None, **kwargs):
        super().__init__(explainer, title='Overview')
        self.name = snp[snp.Symbol == ticker].Security.values[0]
        self.sector = snp[snp.Symbol == ticker]['GICS Sector'].values[0]
        self.industry = snp[snp.Symbol == ticker]['GICS Sub-Industry'].values[0]
        self.hq = snp[snp.Symbol == ticker]['Headquarters Location'].values[0]
        self.founded = snp[snp.Symbol == ticker].Founded.values[0]
        
    def layout(self):
        return dbc.Container([
            dbc.Row([
                dbc.Col([
                    html.H1('Microsoft (MSFT)'),
                    html.Table(
                        children=[
                            html.Tr([
                                html.Td(html.B('Sector:'), style={'marginRight': '20px'}),
                                html.Td(self.sector),
                                html.Td(html.B('Headquarters:'), style={'marginRight': '20px'}),
                                html.Td(self.hq)
                            ]),
                            html.Tr([
                                html.Td(html.B('Industry:'), style={'marginRight': '20px'}),
                                html.Td(self.industry),
                                html.Td(html.B('Founded:'), style={'marginRight': '20px'}),
                                html.Td(self.founded)
                            ])
                        ], 
                        style={'border-collapse': 'separate', 
                               'border-spacing': '20px 5px',
                               'margin-left': '-20px'}
                    )
                ])
            ]),
            dbc.Row([
                dbc.Col([
                    dcc.Graph(
                        id='price_history_graph',
                        figure={
                            'data': [{'x': prices.index, 'y': prices.Close, 'type': 'line'}]
                        }
                    )
                ])
            ])
        ])

## Features Tab

In [11]:
class FeaturesTab(ExplainerComponent):
    def __init__(self, explainer, name=None, **kwargs):
        super().__init__(explainer, title='Features')
    
    def layout(self):
        return html.H3('Features explorer here')

## SHAP Tab

In [12]:
class SHAPTab(ExplainerComponent):
    def __init__(self, explainer, name=None, **kwargs):
        super().__init__(explainer, title='SHAP Analysis')
        
        self.feat_imps = ImportancesComponent(explainer,
                                              depth=15,
                                              no_permutations=True,
                                              hide_popout=True)
        
        self.depend = ShapDependenceComponent(explainer, 
                                              hide_popout=True,
                                              hide_outliers=True)
    
    def layout(self):
        return dbc.Row([
            dbc.Col([self.feat_imps.layout()]),
            dbc.Col([self.depend.layout()])
        ])

# Running Explainer Dashboard

In [38]:
db = ExplainerDashboard(explainer,
                        tabs=[OverviewTab, FeaturesTab, SHAPTab],
                        title='Stock KPIs',
                        bootstrap=dbc.themes.FLATLY,
                        header_hide_download=True,
                        hide_poweredby=True)

db.run(port=8050)

Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
Generating layout...
Calculating dependencies...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Starting ExplainerDashboard on http://74.129.178.98:8050
Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app 'explainerdashb

 * Running on all addresses.
 * Running on http://74.129.178.98:8050/ (Press CTRL+C to quit)
74.129.178.98 - - [29/Aug/2021 21:59:19] "GET / HTTP/1.1" 200 -
74.129.178.98 - - [29/Aug/2021 21:59:20] "GET /_dash-dependencies HTTP/1.1" 200 -
74.129.178.98 - - [29/Aug/2021 21:59:20] "GET /assets/favicon.ico?m=1630208771.8202608 HTTP/1.1" 200 -
74.129.178.98 - - [29/Aug/2021 21:59:20] "GET /_dash-layout HTTP/1.1" 200 -
74.129.178.98 - - [29/Aug/2021 21:59:20] "GET /_dash-component-suites/dash_core_components/async-graph.js HTTP/1.1" 200 -
74.129.178.98 - - [29/Aug/2021 21:59:20] "POST /_dash-update-component HTTP/1.1" 200 -
74.129.178.98 - - [29/Aug/2021 21:59:20] "POST /_dash-update-component HTTP/1.1" 200 -
74.129.178.98 - - [29/Aug/2021 21:59:20] "POST /_dash-update-component HTTP/1.1" 204 -
74.129.178.98 - - [29/Aug/2021 21:59:20] "POST /_dash-update-component HTTP/1.1" 200 -
74.129.178.98 - - [29/Aug/2021 21:59:20] "POST /_dash-update-component HTTP/1.1" 200 -
74.129.178.98 - - [29/Aug